<a href="https://colab.research.google.com/github/gustavoramos82/Identifica-o-de-Flor/blob/main/Identifica%C3%A7%C3%A3o_de_Flores_Fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Flores - Fine Tunning

Neste projeto, apartir de dataset do kaggle (que pode ser acessado [aqui](https://www.kaggle.com/datasets/nadyana/flowers)), será feito a nalise de imagem para classificar de acordo com a espécie da flor. Nesse parte será aplicado o fine-tunning.

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from tensorflow import keras
from keras.layers import Dense, Flatten
from tensorflow.keras.applications import vgg19, resnet50, InceptionResNetV2,Xception
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
# Pegando o diretorio da pasta de cada classifcador
bellflower = os.listdir('/content/drive/MyDrive/Datasets/flowers/bellflower')
daisy = os.listdir('/content/drive/MyDrive/Datasets/flowers/daisy')
dandelion = os.listdir('/content/drive/MyDrive/Datasets/flowers/dandelion')


In [ ]:
# fazendo a extração das imagens
x = []
y = []
img_w, img_h = 224, 224

In [ ]:
for i in bellflower:
  image = Image.open(f'/content/drive/MyDrive/Datasets/flowers/bellflower/{i}').convert('RGB')
  image = image.resize((img_w, img_h))
  x.append(np.asarray(image))
  y.append(0)

In [ ]:
for i in daisy:
  image = Image.open(f'/content/drive/MyDrive/Datasets/flowers/daisy/{i}').convert('RGB')
  image = image.resize((img_w, img_h))
  x.append(np.asarray(image))
  y.append(1)

In [ ]:
for i in dandelion:
  image = Image.open(f'/content/drive/MyDrive/Datasets/flowers/dandelion/{i}').convert('RGB')
  image = image.resize((img_w, img_h))
  x.append(np.asarray(image))
  y.append(2)

In [ ]:
x = np.asarray(x)
x = x.astype('float32')
x /= 255
y = np.asarray(y)

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y,test_size=0.20,random_state=10)

In [ ]:
y_treino = to_categorical(y_treino)
y_teste = to_categorical(y_teste)

In [ ]:
model = InceptionResNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False,classifier_activation='relu')
model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)

219070464/219055592 [==============================] - 1s 0us/step


In [ ]:
newTop = Sequential()
newTop.add(Flatten(input_shape=model.output_shape[1:]))
newTop.add(Dense(3, activation='softmax'))

In [ ]:
metricas = ['accuracy',keras.metrics.Precision(name='Pre'),keras.metrics.Recall(name='Rec')]

In [ ]:
model = Model(inputs=model.input, outputs=newTop(model.output))
model.summary()

optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metricas)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [ ]:
batchSize = 100
numberEpochs = 10
Xtrain = x_treino
Ytrain = y_treino

print("Training the model ...")
batches = list(range(0, len(Ytrain), batchSize))
perm = np.random.permutation(len(Ytrain))

errLoss = []
accLoss = []
errLoss.append(1)
accLoss.append(0)

for e in range(0, numberEpochs):
    for b in batches:
        if b + batchSize < len(Ytrain):
            x = Xtrain[perm[b : b + batchSize]]
            y = Ytrain[perm[b : b + batchSize]]
        else:
            x = Xtrain[perm[b : ]]
            y = Ytrain[perm[b : ]]
        loss = model.train_on_batch(x, y)

    print("\tEpoch %i. [Error, Accuracy]: %.15f, %.15f " % (e+1, loss[0], loss[1]))
    errLoss.append(loss[0])
    accLoss.append(loss[1])

    

Training the model ...
	Epoch 1. [Error, Accuracy]: 0.191315338015556, 0.949999988079071 
	Epoch 2. [Error, Accuracy]: 0.287713319063187, 0.925000011920929 
	Epoch 3. [Error, Accuracy]: 1.196479082107544, 0.725000023841858 
	Epoch 4. [Error, Accuracy]: 15.871839523315430, 0.899999976158142 
	Epoch 5. [Error, Accuracy]: 0.070815168321133, 0.975000023841858 
	Epoch 6. [Error, Accuracy]: 0.021496236324310, 0.975000023841858 
	Epoch 7. [Error, Accuracy]: 0.000542692025192, 1.000000000000000 
	Epoch 8. [Error, Accuracy]: 0.009954581037164, 1.000000000000000 
	Epoch 9. [Error, Accuracy]: 0.014536967501044, 1.000000000000000 
	Epoch 10. [Error, Accuracy]: 0.004122793208808, 1.000000000000000 


In [ ]:
print("Testing the model ...")
acc = model.evaluate(x_teste, y_teste, batch_size=batchSize)

Testing the model ...
10/10 [==============================] - 10s 667ms/step - loss: 0.2052 - accuracy: 0.9458 - Pre: 0.9467 - Rec: 0.9438
